# 5. Building with Generative AI: An Introduction to AgentBricks

Databricks is the premier platform for building production-quality Generative AI applications. While AI/ML is a deep topic, this notebook will give you a high-level overview of **AgentBricks**, Databricks' framework and set of capabilities for building AI agents that can reason and use tools to accomplish complex tasks.

## What is an AI Agent?

An AI Agent is a system that can understand a high-level goal, break it down into steps, and use tools (like querying a database, searching the web, or calling an API) to achieve that goal. The video below explains how Databricks makes this possible with your own enterprise data.

[![Video Thumbnail](https://img.youtube.com/vi/a4XFm8tP_Lw/0.jpg)](https://www.youtube.com/watch?v=a4XFm8tP_Lw "Building AI Agents on Databricks")

## Common AI Agent Patterns to Explore

Building a custom AI agent can be complex, so Databricks provides several pre-built patterns and capabilities that you can explore and customize.

--- 

### 1. Knowledge Assistant (Retrieval-Augmented Generation - RAG)

This is the most common pattern. A RAG agent can "chat" with your documents or structured data, providing answers grounded in your proprietary information. This is powered by **Databricks Vector Search**.

📖 **Resources:**
* [Best Practices for RAG on Databricks](https://www.databricks.com/blog/2023/10/25/best-practices-for-llm-evaluation-of-rag-applications.html)
* [Vector Search Documentation](https://docs.databricks.com/en/generative-ai/vector-search.html)

--- 

### 2. Information Extraction Agent

This agent can read unstructured documents (like PDFs, emails, or images) and extract structured data (like invoice numbers, dates, and amounts), saving it to a Delta table for analysis.

📖 **Resource:** [Intelligent Document Processing with Databricks](https://www.databricks.com/solutions/accelerators/intelligent-document-processing)

---

### 3. Custom LLMs (Finetuning)

For advanced use cases, you can use the Finetuning API to adapt an open-source LLM on your private data. This creates a model that is an expert in your specific domain, using your company's terminology and style.

📖 **Resource:** [Finetuning API Documentation](https://docs.databricks.com/en/large-language-models/foundation-model-training/index.html)

---

## Where to Go Next: Explore AgentBricks on GitHub

The best way to get started with building agents is to explore the official **AgentBricks repository on GitHub**. It provides hands-on examples, labs, and tutorials for the patterns discussed above.

### ➡️ [**Click here to visit the AgentBricks GitHub Repository**](https://github.com/databricks/agentbricks)

## Hands-On Code Examples

### Simple RAG Implementation with Vector Search

In [0]:
# Example: Building a simple knowledge assistant with Vector Search
from databricks.vector_search.client import VectorSearchClient
from databricks.sdk import WorkspaceClient
import openai

# Initialize clients
vsc = VectorSearchClient()
workspace = WorkspaceClient()

# Create a vector search index for your documents
def create_knowledge_base(catalog, schema, table_name):
    """
    Create a vector search index for document embeddings
    """
    index = vsc.create_delta_sync_index(
        endpoint_name="your_vector_search_endpoint",
        index_name=f"{catalog}.{schema}.{table_name}_index",
        source_table_name=f"{catalog}.{schema}.{table_name}",
        pipeline_type="TRIGGERED",
        primary_key="id",
        embedding_source_column="text",
        embedding_model_endpoint_name="databricks-bge-large-en"
    )
    return index

# Search for relevant context
def search_knowledge_base(query, index_name, k=5):
    """
    Search for relevant documents based on user query
    """
    results = vsc.similarity_search(
        index_name=index_name,
        query_text=query,
        columns=["text", "metadata"],
        num_results=k
    )
    return results

# Generate response using RAG pattern
def generate_response(user_query, context_docs):
    """
    Generate a response using retrieved context
    """
    context = "\n".join([doc["text"] for doc in context_docs])
    
    prompt = f"""
    Based on the following context, answer the user's question:
    
    Context: {context}
    
    Question: {user_query}
    
    Answer:
    """
    
    # Using Foundation Model API
    response = workspace.serving_endpoints.query(
        name="databricks-llama-2-70b-chat",
        dataframe_records=[{"prompt": prompt}]
    )
    
    return response.predictions[0]

# Example usage
# query = "What are the best practices for data engineering?"
# docs = search_knowledge_base(query, "main.knowledge.docs_index")
# answer = generate_response(query, docs)
# print(answer)

### Function Calling Agent Example

In [0]:
# Example: Agent with function calling capabilities
import json
from typing import List, Dict

class DataAnalysisAgent:
    """
    An agent that can analyze data using various tools
    """
    
    def __init__(self, workspace_client):
        self.workspace = workspace_client
        self.available_functions = {
            "run_sql_query": self.run_sql_query,
            "create_visualization": self.create_visualization,
            "get_table_schema": self.get_table_schema
        }
    
    def run_sql_query(self, query: str) -> Dict:
        """Execute SQL query and return results"""
        try:
            result = spark.sql(query).toPandas()
            return {
                "success": True,
                "data": result.to_dict('records'),
                "row_count": len(result)
            }
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    def create_visualization(self, data_query: str, chart_type: str = "bar") -> Dict:
        """Create a visualization from data"""
        # This would integrate with plotting libraries
        return {
            "success": True,
            "chart_type": chart_type,
            "message": f"Created {chart_type} chart for query: {data_query}"
        }
    
    def get_table_schema(self, table_name: str) -> Dict:
        """Get schema information for a table"""
        try:
            schema = spark.table(table_name).schema
            return {
                "success": True,
                "schema": str(schema),
                "columns": [field.name for field in schema.fields]
            }
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    def process_request(self, user_message: str) -> str:
        """
        Process user request and determine which functions to call
        """
        # This would use an LLM to determine function calls
        # For demo purposes, we'll use simple logic
        
        if "schema" in user_message.lower():
            # Extract table name and get schema
            table_name = "main.dbdemos_pipeline_bike.bike_trips_gold"  # example
            result = self.get_table_schema(table_name)
            return f"Table schema: {result}"
        
        elif "query" in user_message.lower() or "sql" in user_message.lower():
            # Execute a sample query
            query = "SELECT user_type, COUNT(*) as count FROM main.dbdemos_pipeline_bike.bike_trips_gold GROUP BY user_type LIMIT 10"
            result = self.run_sql_query(query)
            return f"Query results: {result}"
        
        else:
            return "I can help you with SQL queries, table schemas, and visualizations. What would you like to explore?"

# Example usage
# agent = DataAnalysisAgent(workspace)
# response = agent.process_request("Show me the schema for the bike trips table")
# print(response)

## Comprehensive Resource Library

### 📚 **Official Documentation**
* [Generative AI on Databricks - Complete Guide](https://docs.databricks.com/en/generative-ai/index.html)
* [Vector Search Documentation](https://docs.databricks.com/en/generative-ai/vector-search.html)
* [Foundation Model APIs](https://docs.databricks.com/en/machine-learning/foundation-models/index.html)
* [Model Serving for AI Applications](https://docs.databricks.com/en/machine-learning/model-serving/index.html)
* [MLflow for GenAI](https://docs.databricks.com/en/mlflow/llms/index.html)

### 🎥 **Video Learning Resources**
* [Building AI Agents on Databricks](https://www.youtube.com/watch?v=a4XFm8tP_Lw)

### 🛠️ **Hands-On Tutorials and Labs**
* [RAG Application Tutorial](https://docs.databricks.com/en/generative-ai/tutorials/ai-cookbook/index.html)
* [Building AI Agents Workshop](https://github.com/databricks/agentbricks)
* [Vector Search Quickstart](https://docs.databricks.com/en/generative-ai/vector-search-quickstart.html)
* [Generative AI Academy Course](https://academy.databricks.com/path/generative-ai-engineer)

### 📖 **Advanced Reading**
* [RAG Best Practices Guide](https://www.databricks.com/blog/2023/10/25/best-practices-for-llm-evaluation-of-rag-applications.html)
* [Enterprise RAG Architecture](https://www.databricks.com/solutions/accelerators/retrieval-augmented-generation)

### 🧠 **Framework Integration**
* [LangChain with Databricks](https://python.langchain.com/docs/integrations/platforms/databricks)
* [Hugging Face on Databricks](https://docs.databricks.com/en/machine-learning/train-model/huggingface.html)

### 💡 **Community Resources**
* [Databricks Community - GenAI](https://community.databricks.com/s/topic/0TO5w00000094LGGAY/generative-ai)
* [Reddit - r/databricks](https://www.reddit.com/r/databricks/)